In [ ]:
from loica import *
import matplotlib.pyplot as plt
import numpy as np
import getpass

In [ ]:
from flapjack import *
#fj = Flapjack(url_base='flapjack.rudge-lab.org:8000')
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

In [ ]:
receiver1_vector = fj.get('vector', name='receiver1')
receiver2_vector = fj.get('vector', name='receiver2')

In [ ]:
study = fj.create('study', name='Loica testing', description='Test study for demonstrating Loica')

In [ ]:
dna = fj.create('dna', name='nor')
vector = fj.create('vector', name='nor', dnas=dna.id)

In [ ]:
network = GeneticNetwork(vector=vector.id[0])

In [ ]:
sfp = fj.create('signal', name='SFP', color='green', description='Simulated fluorescent protein')
reporter = Reporter(name='SFP', color='green', degradation_rate=0, init_concentration=0, signal_id=sfp.id[0])

In [ ]:
network.add_reporter(reporter)

In [ ]:
ahl1 = Supplement(name='AHL1')
repressor1 = Regulator('LacI')
rec1 = Receiver(input=ahl1, output=repressor1, a=0, b=100, K=1, n=2)

ahl2 = Supplement(name='AHL2')
repressor2 = Regulator('TetR')
rec2 = Receiver(input=ahl2, output=repressor2, a=0, b=100, K=1, n=2)

network.add_operators([rec1,rec2])
network.add_regulators([repressor1,repressor2])

The Nor operator represents a device which can be repressed by either of two repressors $r_1$ and $r_2$, and is modeled as follows, where $\phi(r_1, r_2)$ is the output expression rate:

\begin{equation}
    \phi(r_1, r_2)
    =
    \frac{
        \alpha_0 
        + 
        \alpha_1 (\frac{r1}{K_1})^{n_1} 
        + 
        \alpha_2 (\frac{r2}{K_2})^{n_2}
        +
        \alpha_3 (\frac{r1 r2}{K_1 K_2})^{n_1 + n_2}
    }
    {
        1 
        + 
        (\frac{r1}{K_1})^{n_1}
        + 
        (\frac{r2}{K_2})^{n_2}
        +
        (\frac{r1 r2}{K_1 K_2})^{n_1 + n_2}
    }
\end{equation}


In [ ]:
nor = Nor(input=[repressor1, repressor2], output=reporter, alpha=[1,0,0,0], K=[100,1], n=[4,2])

In [ ]:
network.add_operator(nor)

In [ ]:
plt.figure(figsize=(3,3), dpi=150)
network.draw()

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.05, 1, 1, 1)

def biomass(t):
    return gompertz(t, 0.05, 1, 1, 1)
    
metab = SimulatedMetabolism(biomass, growth_rate)

media = fj.create('media', name='loica', description='Simulated loica media')
strain = fj.create('strain', name='loica', description='Loica test strain')

In [ ]:
# Create list of samples    
samples = []
for conc1 in np.append(0, np.logspace(-2, 2, 18)):
    for conc2 in np.append(0, np.logspace(-3, 1, 18)):
        sample = Sample(genetic_network=network, 
                metabolism=metab,
                media=media.id[0],
                strain=strain.id[0])
        # Add AHL to samples at given concentration
        sample.add_supplement(ahl1, conc1)
        sample.add_supplement(ahl2, conc2)
        samples.append(sample)

In [ ]:
biomass_signal = fj.create('signal', name='SOD', description='Simulated OD', color='black')

In [ ]:
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica NOR',
              description='Simulated NOR generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run(nsr=0)

In [ ]:
assay.upload(fj, study.id[0])

In [ ]:
ahl1_id = fj.get('chemical', name='AHL1').id[0]
ahl2_id = fj.get('chemical', name='AHL2').id[0]

In [ ]:
fig = fj.plot(study=study.id,                     
                vector=vector.id,
                signal=sfp.id,
                type='Heatmap',
                analyte1=ahl1_id,
                analyte2=ahl2_id,
                function='Mean Expression',
                biomass_signal=biomass_signal.id[0],
                normalize='None',
                subplots='Signal',
                markers='Vector',
                plot='All data points'
       )
fig

In [ ]:
nor.characterize(fj, 
            receiver1_vector.id,
            receiver2_vector.id, 
            ahl1_id,
            ahl2_id,
            vector.id, 
            media.id, 
            strain.id, 
            sfp.id, 
            biomass_signal.id,
            0
            )

In [ ]:
nor.alpha0, nor.alpha1, nor.alpha2, nor.alpha3

In [ ]:
nor.rep1_K, nor.rep1_n

In [ ]:
nor.rep2_K, nor.rep2_n